In [333]:
import pandas as pd
import numpy as np
import re
import json
import ast

In [227]:
#reading in activity watch data

activity_watch = pd.read_json("activity_watch_2021_04_18.json")


In [228]:
#separating the buckets column as that is where the dictionary of relevant data is

activity_watch = activity_watch['buckets'].apply(pd.Series)
activity_watch

,id,created,name,type,client,hostname,events
aw-stopwatch,aw-stopwatch,2021-02-27T07:11:34.038428+00:00,None,general.stopwatch,aw-webui,unknown,[]
aw-watcher-afk_DESKTOP-TOACMEE,aw-watcher-afk_DESKTOP-TOACMEE,2021-02-27T07:07:20.676248+00:00,None,afkstatus,aw-watcher-afk,DESKTOP-TOACMEE,[{'timestamp': '2021-04-18T09:41:13.362000+00:...
aw-watcher-window_DESKTOP-TOACMEE,aw-watcher-window_DESKTOP-TOACMEE,2021-02-27T07:07:20.499312+00:00,None,currentwindow,aw-watcher-window,DESKTOP-TOACMEE,[{'timestamp': '2021-04-18T14:15:21.485000+00:...


In [229]:
#Further splitting the events column to extract the time stamp, duration and data

activity_watch_split = pd.DataFrame(activity_watch['events'][2])
activity_watch_split

,timestamp,duration,data
0,2021-04-18T14:15:21.485000+00:00,10.812,"{'app': 'msedge.exe', 'title': 'ActivityWatch ..."
1,2021-04-18T14:15:20.283000+00:00,0.000,"{'app': 'aw-qt.exe', 'title': 'aw-qt'}"
2,2021-04-18T14:15:17.916000+00:00,1.196,"{'app': 'explorer.exe', 'title': ''}"
3,2021-04-18T14:15:16.716000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'Oura API and 7..."
4,2021-04-18T14:15:15.525000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'localhost:7777..."
...,...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846,"{'app': 'SearchApp.exe', 'title': 'Search'}"
26719,2021-02-27T15:08:49.183000+00:00,19.718,"{'app': 'msedge.exe', 'title': 'Sponsor @Activ..."
26720,2021-02-27T15:08:24.630000+00:00,23.357,"{'app': 'msedge.exe', 'title': 'ActivityWatch ..."
26721,2021-02-27T15:08:23.428000+00:00,0.000,"{'app': 'msedge.exe', 'title': 'https://github..."


In [230]:
#dropping the data column in a copy of this

activity_watch_split_1 = activity_watch_split.drop(columns=["data"])
activity_watch_split_1

,timestamp,duration
0,2021-04-18T14:15:21.485000+00:00,10.812
1,2021-04-18T14:15:20.283000+00:00,0.000
2,2021-04-18T14:15:17.916000+00:00,1.196
3,2021-04-18T14:15:16.716000+00:00,0.000
4,2021-04-18T14:15:15.525000+00:00,0.000
...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846
26719,2021-02-27T15:08:49.183000+00:00,19.718
26720,2021-02-27T15:08:24.630000+00:00,23.357
26721,2021-02-27T15:08:23.428000+00:00,0.000


In [231]:
#splitting the data column of activity_watch_split, to extract the app and title being used

activity_watch_split_2 = activity_watch_split['data'].apply(pd.Series)
activity_watch_split_2

,app,title
0,msedge.exe,ActivityWatch and 79 more pages - Personal - M...
1,aw-qt.exe,aw-qt
2,explorer.exe,
3,msedge.exe,Oura API and 78 more pages - Personal - Micros...
4,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
...,...,...
26718,SearchApp.exe,Search
26719,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
26720,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
26721,msedge.exe,https://github.com/ActivityWatch and 22 more p...


In [365]:
#combining the two split activity watch dataframes so that it contains both the timestamp, app, title and duration

activity_watch_final = pd.concat([activity_watch_split_1, activity_watch_split_2], axis=1)
activity_watch_final

,timestamp,duration,app,title
0,2021-04-18T14:15:21.485000+00:00,10.812,msedge.exe,ActivityWatch and 79 more pages - Personal - M...
1,2021-04-18T14:15:20.283000+00:00,0.000,aw-qt.exe,aw-qt
2,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
3,2021-04-18T14:15:16.716000+00:00,0.000,msedge.exe,Oura API and 78 more pages - Personal - Micros...
4,2021-04-18T14:15:15.525000+00:00,0.000,msedge.exe,localhost:7777/callback?code=AQAAFYrVKDTnt6XUV...
...,...,...,...,...
26718,2021-02-27T15:09:10.151000+00:00,4.846,SearchApp.exe,Search
26719,2021-02-27T15:08:49.183000+00:00,19.718,msedge.exe,Sponsor @ActivityWatch on GitHub Sponsors and ...
26720,2021-02-27T15:08:24.630000+00:00,23.357,msedge.exe,ActivityWatch and 22 more pages - Personal - M...
26721,2021-02-27T15:08:23.428000+00:00,0.000,msedge.exe,https://github.com/ActivityWatch and 22 more p...


Here I will be checking for any duplicates in the timestamp column, and any anomalies in the duration column. I will also be going over any nulls in the app and title column.

In [267]:
#checking for nulls in the title column that may not have been labelled as null

activity_watch_nulls = activity_watch_final[activity_watch_final['title'] == ""]
activity_watch_nulls


,timestamp,duration,app,title
2,2021-04-18T14:15:17.916000+00:00,1.196,explorer.exe,
60,2021-04-18T14:04:11.987000+00:00,0.000,explorer.exe,
85,2021-04-18T14:00:06.266000+00:00,0.000,explorer.exe,
87,2021-04-18T14:00:00.310000+00:00,2.361,explorer.exe,
129,2021-04-18T13:47:31.655000+00:00,0.000,explorer.exe,
...,...,...,...,...
26648,2021-02-27T15:23:45.428000+00:00,0.100,explorer.exe,
26684,2021-02-27T15:15:53.395000+00:00,0.000,explorer.exe,
26688,2021-02-27T15:15:44.524000+00:00,0.027,explorer.exe,
26692,2021-02-27T15:15:38.558000+00:00,0.042,explorer.exe,


In [271]:
#checking if these null columns have any useful value in the app column

activity_watch_nulls["app"].value_counts()

explorer.exe                                                      1430
unknown                                                            479
msedge.exe                                                         110
msrdc.exe                                                           48
CredentialUIBroker.exe                                              14
LockApp.exe                                                         11
dwm.exe                                                              8
Zoom.exe                                                             5
Code - Insiders.exe                                                  3
Code.exe                                                             2
OpenWith.exe                                                         2
CodeSetup-insider-0640cdeb8ba34f2f00cf4de73783632c4a692c52.tmp       2
WINWORD.EXE                                                          2
ScreenClippingHost.exe                                               2
rundll

The majority of it is unlabelled browsing on internet browser. We have two options:

Option 1) Drop all these empty columns
Option 2) Keep these empty columns

This will depend on how much information the original "app" column gives us, so I will be checking that first.

In [274]:
activity_watch_final["app"].value_counts()[0:15]

msedge.exe                  15235
explorer.exe                 2907
Code.exe                     1656
unknown                      1437
msrdc.exe                    1304
Code - Insiders.exe          1190
LockApp.exe                   573
notepad.exe                   372
msrdcw.exe                    269
ApplicationFrameHost.exe      258
WindowsTerminal.exe           240
ShellExperienceHost.exe       213
Zoom.exe                      181
CredentialUIBroker.exe        140
Spotify.exe                   140
Name: app, dtype: int64

Given that the majority of apps are msedge and explorer, and the null "app" values represent 9% and 16% of the original "app" column, it is best that we keep this data, as they are statistically significant.

Now we will be checking for duplicates in the timestamp column.

In [291]:
#extracting just the duplicates in the timestamp column

activity_watch_final_duplicates = activity_watch_final[activity_watch_final.duplicated(subset=["timestamp"]) == True]
activity_watch_final_duplicates["timestamp"].value_counts()



2021-03-14T14:20:57.804000+00:00    31
2021-03-14T14:51:52.157000+00:00    15
2021-03-14T14:49:35.283000+00:00    12
2021-02-28T15:36:17.657000+00:00     2
2021-04-15T05:00:01.575000+00:00     2
                                    ..
2021-02-27T16:32:40.298000+00:00     1
2021-02-27T17:34:33.950000+00:00     1
2021-02-28T01:50:25.705000+00:00     1
2021-02-28T03:09:36.032000+00:00     1
2021-03-01T01:26:58.983000+00:00     1
Name: timestamp, Length: 437, dtype: int64

In [294]:
#diving deeper into the duplicates to see if other variables such as "app" and "title" are the same

activity_watch_final_duplicates[0:31]

,timestamp,duration,app,title
5037,2021-04-15T05:00:01.575000+00:00,94.649,msedge.exe,API and 16 more pages - Personal - Microsoft​ ...
5038,2021-04-15T05:00:01.575000+00:00,104.949,msedge.exe,API and 16 more pages - Personal - Microsoft​ ...
19279,2021-03-14T14:51:52.157000+00:00,153.135,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19280,2021-03-14T14:51:52.157000+00:00,142.890,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19281,2021-03-14T14:51:52.157000+00:00,132.719,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19282,2021-03-14T14:51:52.157000+00:00,122.521,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19283,2021-03-14T14:51:52.157000+00:00,112.302,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19284,2021-03-14T14:51:52.157000+00:00,102.091,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19285,2021-03-14T14:51:52.157000+00:00,91.887,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...
19286,2021-03-14T14:51:52.157000+00:00,81.695,Code - Insiders.exe,● scratch.ipynb - braindump - Visual Studio Co...


Upon further inspection it looks as though that duplicate pairs contain the same information across all columns, apart from the "duration" column. I will be taking the average duration for each timestamp to ensure I don't lose any information.

In [368]:
#seeing the number of unique names in activity watch column app

activity_watch_new["app"].value_counts()[30:50]

CodeSetup-insider-0640cdeb8ba34f2f00cf4de73783632c4a692c52.tmp    6
inno_updater.exe                                                  5
CodeSetup-stable-3c4e3df9e89829dce27b7b5c24508306b151f30d.tmp     5
Notion.exe                                                        5
rundll32.exe                                                      4
CodeSetup-stable-f30a9b73e8ffc278e71575118b6bf568f04587c8.tmp     4
PickerHost.exe                                                    4
msiexec.exe                                                       3
dccw.exe                                                          3
CodeSetup-stable-c185983a683d14c396952dd432459097bc7f757f.tmp     3
StartMenuExperienceHost.exe                                       2
CodeSetup-insider-9b2ee7fb7d970b9a628fdb3545bdd01d67078d16.tmp    2
OpenWith.exe                                                      2
CodeSetup-insider-e590188f17162393f50feec19263398e6fe02d13.tmp    2
spotify_installer-1.1.54.592.gc0b20638-29.exe   

In [236]:
#reading spotify data in

spotify = pd.read_csv("spotify_history_ore_2021_04_18.csv")
spotify.columns

Index(['timestamp', 'name', 'artists', 'uri', 'duration_ms', 'explicit'], dtype='object')

In [338]:
#reading google calendar data in

g_calendar = pd.read_csv("google_calendar_ore_2021_04_18.csv")
g_calendar.columns

Index(['kind', 'etag', 'id', 'status', 'htmlLink', 'created', 'updated',
       'summary', 'creator', 'organizer', 'start', 'end', 'recurringEventId',
       'originalStartTime', 'visibility', 'iCalUID', 'sequence', 'reminders',
       'eventType', 'description', 'colorId', 'transparency', 'location',
       'attendees', 'guestsCanInviteOthers', 'privateCopy', 'source',
       'hangoutLink', 'conferenceData', 'extendedProperties',
       'guestsCanModify', 'guestsCanSeeOtherGuests'],
      dtype='object')

In [360]:
#inspecting the data to identify any potential anomalies
g_calendar["created"].unique()

array(['2015-04-04T02:15:49.000Z', '2016-04-01T01:56:31.000Z',
       '2020-10-19T15:35:00.000Z', '2020-10-19T15:34:38.000Z',
       '2020-10-19T15:34:39.000Z', '2020-12-17T16:17:18.000Z',
       '2020-12-13T19:54:36.000Z', '2020-12-25T20:07:28.000Z',
       '2020-12-26T14:36:21.000Z', '2020-12-26T20:06:11.000Z',
       '2020-12-23T23:29:59.000Z', '2020-12-31T20:26:42.000Z',
       '2021-01-01T01:12:06.000Z', '2021-01-01T10:22:12.000Z',
       '2020-12-21T01:50:28.000Z', '2021-01-04T22:56:34.000Z',
       '2021-01-06T15:56:23.000Z', '2021-01-10T20:33:09.000Z',
       '2021-01-12T10:00:46.000Z', '2021-01-14T17:55:48.000Z',
       '2021-01-15T06:34:48.000Z', '2020-12-21T18:30:05.000Z',
       '2021-01-13T15:53:47.000Z', '2021-01-15T18:00:44.000Z',
       '2021-01-12T08:52:04.000Z', '2021-01-17T23:38:48.000Z',
       '2021-01-18T15:14:06.000Z', '2020-12-23T20:39:07.000Z',
       '2021-01-22T06:10:43.000Z', '2021-01-22T09:05:15.000Z',
       '2021-01-21T10:09:12.000Z', '2021-01-25T02:27:55

In [315]:
#inspecting the data to identify any potential anomalies
g_calendar["location"].value_counts()

Google Meet (instructions in description)                                                 59
https://zoom.us/j/92547005201                                                              1
https://zoom.us/j/97477945016?pwd=c1FLMWJjZDVucnFGY2k1MXV4MHpDUT09                         1
Accra ACC                                                                                  1
Tribeca Hotel, Accra                                                                       1
Schiphol Airport (Evert v/d Beekstraat 202, 1118 Amsterdam North Holland, Netherlands)     1
https://meet.google.com/kgk-qgha-gai                                                       1
Kotoka International Airport (Accra Accra, Ghana)                                          1
https://us02web.zoom.us/j/85090663120?pwd=NDhjRWt5UCtabGVkOTVxYm9lakN0QT09                 1
https://us02web.zoom.us/j/84101445821                                                      1
https://meet.google.com/vra-emhm-zoo                                  

In [325]:
#inspecting the data to identify any potential anomalies
type(g_calendar["start"][0])

str

The columns start and end both contain dictionaries, so I will be separating them into dataframes then recombining.

In [329]:
#As discovered above, the dictionaries have been stored as strings. I will need to convert them into dictionaries so that I can split the columns.
g_calendar_start = g_calendar["start"].apply(ast.literal_eval)
g_calendar_start

0      {'dateTime': '2021-04-14T16:00:00-07:00', 'tim...
1                                 {'date': '2021-03-15'}
2              {'dateTime': '2021-01-11T04:00:00-08:00'}
3      {'dateTime': '2021-01-10T14:15:00-08:00', 'tim...
4      {'dateTime': '2021-01-11T04:00:00-08:00', 'tim...
                             ...                        
252            {'dateTime': '2021-04-17T12:15:00-07:00'}
253            {'dateTime': '2021-04-17T18:15:00-07:00'}
254            {'dateTime': '2021-04-17T20:30:00-07:00'}
255            {'dateTime': '2021-04-17T21:00:00-07:00'}
256            {'dateTime': '2021-04-17T21:00:00-07:00'}
Name: start, Length: 257, dtype: object

In [332]:
#The start times for google calendar events
g_calendar_start = g_calendar_start.apply(pd.Series)
g_calendar_start.rename(columns={"dateTime": "start_dateTime"}, inplace=True)
g_calendar_start

,start_dateTime,timeZone,date
0,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-15
2,2021-01-11T04:00:00-08:00,NaN,NaN
3,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN
4,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN
...,...,...,...
252,2021-04-17T12:15:00-07:00,NaN,NaN
253,2021-04-17T18:15:00-07:00,NaN,NaN
254,2021-04-17T20:30:00-07:00,NaN,NaN
255,2021-04-17T21:00:00-07:00,NaN,NaN


In [334]:
#splitting google calendar end date
g_calendar_end = g_calendar["end"].apply(ast.literal_eval)
g_calendar_end = g_calendar_end.apply(pd.Series)
g_calendar_end.rename(columns={"dateTime": "end_dateTime"}, inplace=True)
g_calendar_end

,end_dateTime,timeZone,date
0,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-16
2,2021-01-11T13:55:00-08:00,NaN,NaN
3,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...
252,2021-04-17T18:00:00-07:00,NaN,NaN
253,2021-04-17T20:15:00-07:00,NaN,NaN
254,2021-04-17T21:00:00-07:00,NaN,NaN
255,2021-04-17T22:15:00-07:00,NaN,NaN


from looking at these different datetimes, it looks as though it is in UTC minus X hours, depending on the timezone it is in.

In [335]:
#combining both the start and end columns into one dataframe

g_calendar_times_combined = g_calendar_start.join(g_calendar_end, lsuffix='_act', rsuffix='_res')
g_calendar_times_combined

,start_dateTime,timeZone_act,date_act,end_dateTime,timeZone_res,date_res
0,2021-04-14T16:00:00-07:00,Europe/Dublin,NaN,2021-04-14T17:00:00-07:00,Europe/Dublin,NaN
1,NaN,NaN,2021-03-15,NaN,NaN,2021-03-16
2,2021-01-11T04:00:00-08:00,NaN,NaN,2021-01-11T13:55:00-08:00,NaN,NaN
3,2021-01-10T14:15:00-08:00,Atlantic/Reykjavik,NaN,2021-01-10T20:50:00-08:00,Europe/Berlin,NaN
4,2021-01-11T04:00:00-08:00,Europe/Berlin,NaN,2021-01-11T13:55:00-08:00,America/Vancouver,NaN
...,...,...,...,...,...,...
252,2021-04-17T12:15:00-07:00,NaN,NaN,2021-04-17T18:00:00-07:00,NaN,NaN
253,2021-04-17T18:15:00-07:00,NaN,NaN,2021-04-17T20:15:00-07:00,NaN,NaN
254,2021-04-17T20:30:00-07:00,NaN,NaN,2021-04-17T21:00:00-07:00,NaN,NaN
255,2021-04-17T21:00:00-07:00,NaN,NaN,2021-04-17T22:15:00-07:00,NaN,NaN


In [376]:
#combining the calendar times dataframe with the original dataframe

g_calendar_final = g_calendar.join(g_calendar_times_combined, lsuffix='_act', rsuffix='_res')
g_calendar_final["description"].value_counts()[0:5]

Check in on progress made. Set milestone for next sync                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        62
Hi,\n\nScheduling a recurring sync for your VR Aspirations track!\n\nMeeting Agenda:\n- Update on what you’ve been done since last sync\n   - Review progress\n   - Discuss blockers\n   - What can Ore can I do to help \n\n- Talk about what you plan to do next.\n\n- Misc. (anything else that top of mind for you)

In [ ]:
#here I will be dropping the irrelevant columns

Based on the results above I will only be keeping the following columns as they provide us with the most information:

- meeting start time (g_calendar_start)
- meeting end time (g_clanedar_end)
- timezone_act
- created
- summary
- description

In [238]:
#reading oura file in

oura_file = open('oura_data_ore_2021_04_18.json')
oura_json = oura_file.read()
oura_data = json.loads(oura_json)


In [240]:
#splitting oura_data into first dataframe which just has activity data. I will be combining it with other oura dataframes on the summary_date column as that column is in all dataframes

oura_activity = pd.DataFrame.from_records(oura_data["activity"])
oura_activity
oura_activity["summary_date"].describe()

count             84
unique            84
top       2021-02-12
freq               1
Name: summary_date, dtype: object

In [241]:
#extracting restufl periods into it's own dataframe

oura_restful_periods = pd.DataFrame.from_records(oura_data["restful_periods"])
oura_restful_periods
oura_restful_periods["summary_date"].describe()

count             52
unique            39
top       2021-02-28
freq               3
Name: summary_date, dtype: object

In [255]:
#extracting readiness into it's own dataframe

oura_readiness = pd.DataFrame.from_records(oura_data["readiness"])
type(oura_readiness["summary_date"][5])
oura_readiness

,period_id,rest_mode_state,score,score_activity_balance,score_hrv_balance,score_previous_day,score_previous_night,score_recovery_index,score_resting_hr,score_sleep_balance,score_temperature,summary_date
0,0,0,87,0,0,0,71,100,95,0,99,2021-01-21
1,1,0,73,0,0,90,62,48,90,0,100,2021-01-22
2,1,0,87,0,0,89,82,100,93,80,90,2021-01-23
3,2,0,81,92,0,93,70,81,95,70,98,2021-01-24
4,0,0,51,59,0,74,32,7,82,50,97,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...
77,1,0,47,36,70,87,39,57,50,19,92,2021-04-13
78,0,0,44,31,64,51,64,64,27,25,88,2021-04-14
79,1,0,54,40,66,80,46,74,76,25,100,2021-04-15
80,0,0,46,52,69,68,26,41,68,21,90,2021-04-16


In [243]:
#extracting sleep into it's own dataframe

oura_sleep = pd.DataFrame.from_records(oura_data["sleep"])
oura_sleep["summary_date"][3]

'2021-01-24'

In [258]:
#left joining datarame on the summary column first
oura_combined = oura_activity.join(oura_restful_periods, lsuffix='_act', rsuffix='_res')
oura_combined

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,bedtime_end,bedtime_start,breath_average,duration,period_id,summary_date_res,timezone_res,hr_average,hr_lowest,rmssd
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,2021-01-21T07:36:24-08:00,2021-01-21T07:23:24-08:00,15.250,780.0,1.0,2021-01-21,-480.0,NaN,NaN,NaN
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,2021-01-23T13:25:29-08:00,2021-01-23T13:09:29-08:00,15.125,960.0,3.0,2021-01-23,-480.0,59.00,58.0,64.0
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,2021-01-23T14:50:29-08:00,2021-01-23T14:39:29-08:00,15.125,660.0,4.0,2021-01-23,-480.0,60.00,60.0,NaN
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,2021-01-28T15:34:12-08:00,2021-01-28T15:06:12-08:00,15.250,1680.0,1.0,2021-01-28,-480.0,85.67,83.0,29.0
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,2021-01-30T13:01:59-08:00,2021-01-30T12:40:59-08:00,14.875,1260.0,1.0,2021-01-30,-480.0,65.75,64.0,63.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [259]:
#combining the readiness dataframe to the above combined dataframe
oura_combined_1 = oura_combined.join(oura_readiness, lsuffix='_act_1', rsuffix='_res_1')
oura_combined_1

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_res_1,score_activity_balance,score_hrv_balance,score_previous_day,score_previous_night,score_recovery_index,score_resting_hr,score_sleep_balance,score_temperature,summary_date
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,87.0,0.0,0.0,0.0,71.0,100.0,95.0,0.0,99.0,2021-01-21
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,73.0,0.0,0.0,90.0,62.0,48.0,90.0,0.0,100.0,2021-01-22
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,87.0,0.0,0.0,89.0,82.0,100.0,93.0,80.0,90.0,2021-01-23
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,81.0,92.0,0.0,93.0,70.0,81.0,95.0,70.0,98.0,2021-01-24
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,51.0,59.0,0.0,74.0,32.0,7.0,82.0,50.0,97.0,2021-01-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,54.0,40.0,66.0,80.0,46.0,74.0,76.0,25.0,100.0,2021-04-15
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,46.0,52.0,69.0,68.0,26.0,41.0,68.0,21.0,90.0,2021-04-16
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,35.0,51.0,57.0,86.0,40.0,23.0,1.0,17.0,96.0,2021-04-17
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [260]:
#combining the readiness dataframe to the above combined dataframe, to make the full oura dataframe
oura_test_combined_final = oura_combined_1.join(oura_sleep, lsuffix='_act_1', rsuffix='_res_1')
oura_test_combined_final

,average_met,cal_active,cal_total,class_5min,daily_movement,day_end,day_start,high,inactive,inactivity_alerts,...,score_efficiency,score_latency,score_rem,score_total,summary_date_res_1,temperature_delta,temperature_deviation,timezone,total_res_1,temperature_trend_deviation
0,0.93750,23,1942,0000000000000000000000000000000000000000000000...,461,2021-01-21T03:59:59-08:00,2021-01-20T04:00:00-08:00,0,141,1,...,86.0,91.0,63.0,84.0,2021-01-21,-0.15,-0.15,-480.0,28830.0,NaN
1,1.34375,234,2311,2232332222222221222222322222333332322223311221...,4151,2021-01-22T03:59:59-08:00,2021-01-21T04:00:00-08:00,1,792,1,...,83.0,67.0,39.0,55.0,2021-01-22,-0.10,-0.10,-480.0,21420.0,0.01
2,1.34375,290,2350,1211222221111111122212211222222222222224434333...,5337,2021-01-23T03:59:59-08:00,2021-01-22T04:00:00-08:00,0,612,0,...,69.0,97.0,86.0,82.0,2021-01-23,-0.32,-0.32,-480.0,28410.0,0.01
3,1.15625,96,2048,2232222211122211111111111111111111112211111111...,1832,2021-01-24T03:59:59-08:00,2021-01-23T04:00:00-08:00,0,517,1,...,86.0,86.0,66.0,61.0,2021-01-24,-0.26,-0.26,-480.0,22890.0,0.02
4,1.68750,707,2894,1111111111111111111111221212000000000221111112...,13257,2021-01-25T03:59:59-08:00,2021-01-24T04:00:00-08:00,2,703,0,...,76.0,91.0,72.0,69.0,2021-01-27,0.16,0.16,-480.0,25020.0,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79,1.65625,721,2859,1111112112111112222222222222222323222222212211...,12814,2021-04-14T03:59:59-07:00,2021-04-13T04:00:00-07:00,20,710,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
80,1.40625,301,2400,1111111111111111111112111111111111111121121121...,5541,2021-04-15T03:59:59-07:00,2021-04-14T04:00:00-07:00,0,774,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
81,1.37500,193,2324,1111111111111112111112221222222222233333222222...,3148,2021-04-16T03:59:59-07:00,2021-04-15T04:00:00-07:00,1,1062,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
82,1.50000,550,2609,1111111111111111122111111111111111111221122211...,10674,2021-04-17T03:59:59-07:00,2021-04-16T04:00:00-07:00,2,451,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [363]:
#inspecting what columns to drop
oura_test_combined_final.columns

Index(['average_met', 'cal_active', 'cal_total', 'class_5min',
       'daily_movement', 'day_end', 'day_start', 'high', 'inactive',
       'inactivity_alerts', 'low', 'medium', 'met_1min', 'met_min_high',
       'met_min_inactive', 'met_min_low', 'met_min_medium', 'non_wear', 'rest',
       'rest_mode_state_act_1', 'score_act_1', 'score_meet_daily_targets',
       'score_move_every_hour', 'score_recovery_time', 'score_stay_active',
       'score_training_frequency', 'score_training_volume', 'steps',
       'summary_date_act', 'target_calories', 'target_km', 'target_miles',
       'timezone_act', 'to_target_km', 'to_target_miles', 'total_act_1',
       'bedtime_end_act_1', 'bedtime_start_act_1', 'breath_average_act_1',
       'duration_act_1', 'period_id_act_1', 'summary_date_res', 'timezone_res',
       'hr_average_act_1', 'hr_lowest_act_1', 'rmssd_act_1', 'period_id_res_1',
       'rest_mode_state_res_1', 'score_res_1', 'score_activity_balance',
       'score_hrv_balance', 'score_prev